## Install a pip package in the current Jupyter kernel

In [0]:
import sys
!{sys.executable} -m pip install pokebase

## Import PokeAPI wrapper library for  Python 3

In [0]:
import pokebase as pb

## Quick lookup

In [0]:
rayquaza = pb.pokemon('rayquaza')
print(rayquaza.height)
print(rayquaza.weight)


70
2065


In [0]:
rayquaza.types

[{'slot': 2, 'type': {'name': 'flying', 'url': 'https://pokeapi.co/api/v2/type/3/'}},
 {'slot': 1, 'type': {'name': 'dragon', 'url': 'https://pokeapi.co/api/v2/type/16/'}}]

In [0]:
rayquaza.stats

[{'base_stat': 95, 'effort': 0, 'stat': {'name': 'speed', 'url': 'https://pokeapi.co/api/v2/stat/6/'}},
 {'base_stat': 90, 'effort': 0, 'stat': {'name': 'special-defense', 'url': 'https://pokeapi.co/api/v2/stat/5/'}},
 {'base_stat': 150, 'effort': 1, 'stat': {'name': 'special-attack', 'url': 'https://pokeapi.co/api/v2/stat/4/'}},
 {'base_stat': 90, 'effort': 0, 'stat': {'name': 'defense', 'url': 'https://pokeapi.co/api/v2/stat/3/'}},
 {'base_stat': 150, 'effort': 2, 'stat': {'name': 'attack', 'url': 'https://pokeapi.co/api/v2/stat/2/'}},
 {'base_stat': 105, 'effort': 0, 'stat': {'name': 'hp', 'url': 'https://pokeapi.co/api/v2/stat/1/'}}]

## Fetch txt file from Smogon

In [0]:
from urllib.request import urlopen
import json

data = urlopen("https://www.smogon.com/stats/2019-03/gen7ubers-0.txt")
usage = []
for line in data:
  decoded = line.decode('utf-8')
  decoded = decoded.replace(" ", "")
  if decoded.startswith('|'):
    usage.append(decoded.split('|')[1:-1])

In [0]:
usage

[['Rank', 'Pokemon', 'Usage%', 'Raw', '%', 'Real', '%'],
 ['1',
  'Groudon-Primal',
  '39.06828%',
  '274400',
  '39.068%',
  '216909',
  '42.973%'],
 ['2', 'Xerneas', '28.30899%', '198831', '28.309%', '135823', '26.909%'],
 ['3',
  'Necrozma-Dusk-Mane',
  '21.22046%',
  '149044',
  '21.220%',
  '106346',
  '21.069%'],
 ['4', 'Yveltal', '20.95863%', '147205', '20.959%', '114685', '22.721%'],
 ['5', 'Kyogre-Primal', '20.72441%', '145560', '20.724%', '104192', '20.642%'],
 ['6', 'Marshadow', '20.52509%', '144160', '20.525%', '99669', '19.746%'],
 ['7', 'Arceus', '12.14491%', '85301', '12.145%', '60533', '11.993%'],
 ['8', 'Zygarde', '10.21769%', '71765', '10.218%', '51311', '10.166%'],
 ['9', 'Lugia', '9.96013%', '69956', '9.960%', '48418', '9.592%'],
 ['10', 'Aegislash', '9.79626%', '68805', '9.796%', '48596', '9.628%'],
 ['11', 'Ho-Oh', '8.58036%', '60265', '8.580%', '45480', '9.010%'],
 ['12', 'Salamence-Mega', '7.97611%', '56021', '7.976%', '38780', '7.683%'],
 ['13', 'Blaziken-Mega'

## Use pandas to visualize Smogon competitive usage data

In [0]:
import pandas as pd

In [0]:
smogon_df = pd.DataFrame(usage[1:], columns=usage[0])
smogon_df

# usage - weighted based on matchmaking rating
# raw - unweighted usage (on team)
# real - actually used in battle

,Rank,Pokemon,Usage%,Raw,%,Real,%
0,1,Groudon-Primal,39.06828%,274400,39.068%,216909,42.973%
1,2,Xerneas,28.30899%,198831,28.309%,135823,26.909%
2,3,Necrozma-Dusk-Mane,21.22046%,149044,21.220%,106346,21.069%
3,4,Yveltal,20.95863%,147205,20.959%,114685,22.721%
4,5,Kyogre-Primal,20.72441%,145560,20.724%,104192,20.642%
5,6,Marshadow,20.52509%,144160,20.525%,99669,19.746%
6,7,Arceus,12.14491%,85301,12.145%,60533,11.993%
7,8,Zygarde,10.21769%,71765,10.218%,51311,10.166%
8,9,Lugia,9.96013%,69956,9.960%,48418,9.592%
9,10,Aegislash,9.79626%,68805,9.796%,48596,9.628%


## Obtain list of legendary and mystical pokemon from Bulbapedia

In [0]:
import requests
from bs4 import BeautifulSoup

# scrape list of legendary and mystical pokemon from bulbapedia
page = requests.get("https://bulbapedia.bulbagarden.net/wiki/Legendary_Pok%C3%A9mon")
soup = BeautifulSoup(page.content, 'html.parser')

# list of <a> tags of legendary pokemon
a_list = soup.select('td a[title*=(Pokémon)]')

# extract text from <a> tags
legend_list = [lp.get_text() for lp in a_list]

# data cleaning: replace whitespace with hyphen, remove colon
legend_list = [p.lower().replace(' ', '-').replace(':', '') for p in legend_list]
legend_list

['articuno',
 'zapdos',
 'moltres',
 'mewtwo',
 'mew',
 'raikou',
 'entei',
 'suicune',
 'lugia',
 'ho-oh',
 'celebi',
 'regirock',
 'regice',
 'registeel',
 'latias',
 'latios',
 'kyogre',
 'groudon',
 'rayquaza',
 'jirachi',
 'deoxys',
 'uxie',
 'mesprit',
 'azelf',
 'dialga',
 'palkia',
 'heatran',
 'regigigas',
 'giratina',
 'cresselia',
 'phione',
 'manaphy',
 'darkrai',
 'shaymin',
 'arceus',
 'victini',
 'cobalion',
 'terrakion',
 'virizion',
 'tornadus',
 'thundurus',
 'reshiram',
 'zekrom',
 'landorus',
 'kyurem',
 'keldeo',
 'meloetta',
 'genesect',
 'xerneas',
 'yveltal',
 'zygarde',
 'diancie',
 'hoopa',
 'volcanion',
 'type-null',
 'silvally',
 'tapu-koko',
 'tapu-lele',
 'tapu-bulu',
 'tapu-fini',
 'cosmog',
 'cosmoem',
 'solgaleo',
 'lunala',
 'necrozma',
 'magearna',
 'marshadow',
 'zeraora',
 'meltan',
 'melmetal']

## After scraping and cleaning, test for validity by searching for each name in the API

In [0]:
for p in legend_list:
  try:
    pb.pokemon(p)
  except ValueError:
    print("Cannot find", p)

Cannot find deoxys
Cannot find giratina
Cannot find shaymin
Cannot find tornadus
Cannot find thundurus
Cannot find landorus
Cannot find keldeo
Cannot find meloetta
Cannot find meltan
Cannot find melmetal


## The above Pokemon have alternate forms and we must handle each manually

P.S. Meltan and Melmetal haven't been added to the database at the moment. https://github.com/PokeAPI/pokeapi/issues/414

In [0]:
# deoxys: deoxys-normal, deoxys-attack, deoxys-defense, deoxys-speed
try:
  legend_list.remove('deoxys')
  legend_list.extend(['deoxys-normal', 'deoxys-attack', 'deoxys-defense', 'deoxys-speed'])
except ValueError:
  print('Error')

# giratina: giratina-altered, giratina-origin
try:
  legend_list.remove('giratina')
  legend_list.extend(['giratina-altered', 'giratina-origin'])
except ValueError:
  print('Error')

# shaymin: shaymin-land, shaymin-sky
try:
  legend_list.remove('shaymin')
  legend_list.extend(['shaymin-land', 'shaymin-sky'])
except ValueError:
  print('Error')

# tornadus: tornadus-incarnate, tornadus-therian
try:
  legend_list.remove('tornadus')
  legend_list.extend(['tornadus-incarnate', 'tornadus-therian'])
except ValueError:
  print('Error')

# thundurus: thundurus-incarnate, thundurus-therian
try:
  legend_list.remove('thundurus')
  legend_list.extend(['thundurus-incarnate', 'thundurus-therian'])
except ValueError:
  print('Error')

# landorus: landorus-incarnate, landorus-therian
try:
  legend_list.remove('landorus')
  legend_list.extend(['landorus-incarnate', 'landorus-therian'])
except ValueError:
  print('Error')

# keldeo: keldeo-ordinary, keldeo-resolute
try:
  legend_list.remove('keldeo')
  legend_list.extend(['keldeo-ordinary', 'keldeo-resolute'])
except ValueError:
  print('Error')

# meloetta: meloetta-aria, meloetta-pirouette
try:
  legend_list.remove('meloetta')
  legend_list.extend(['meloetta-aria', 'meloetta-pirouette'])
except ValueError:
  print('Error')

# meltan: remove for now
try:
  legend_list.remove('meltan')
except ValueError:
  print('Error')

# melmetal: remove for now
try:
  legend_list.remove('melmetal')
except ValueError:
  print('Error')

## Test again for validity

In [0]:
for p in legend_list:
  try:
    pb.pokemon(p)
  except ValueError:
    print("Cannot find", p)

## Obtain list of types

In [0]:
# scrape list of legendary and mystical pokemon from bulbapedia
page = requests.get("https://bulbapedia.bulbagarden.net/wiki/Type")
soup = BeautifulSoup(page.content, 'html.parser')

# list of <a> tags of types
a_list = soup.select('td a[title*=(type)]')

# extract text from <a> tags
type_list = [lp.get_text() for lp in a_list]

# ignore ??? type
type_list = type_list[:-1]
type_list

In [0]:
pstat = []
for a in legend_list:
  pstat.append()
  
  
  
  
rayquaza = pb.pokemon('rayquaza')
print(rayquaza.height)
print(rayquaza.weight)
  

articuno
zapdos
moltres
mewtwo
mew
raikou
entei
suicune
lugia
ho-oh
celebi
regirock
regice
registeel
latias
latios
kyogre
groudon
rayquaza
jirachi
uxie
mesprit
azelf
dialga
palkia
heatran
regigigas
cresselia
phione
manaphy
darkrai
arceus
victini
cobalion
terrakion
virizion
reshiram
zekrom
kyurem
genesect
xerneas
yveltal
zygarde
diancie
hoopa
volcanion
type-null
silvally
tapu-koko
tapu-lele
tapu-bulu
tapu-fini
cosmog
cosmoem
solgaleo
lunala
necrozma
magearna
marshadow
zeraora
deoxys-normal
deoxys-attack
deoxys-defense
deoxys-speed
giratina-altered
giratina-origin
shaymin-land
shaymin-sky
tornadus-incarnate
tornadus-therian
thundurus-incarnate
thundurus-therian
landorus-incarnate
landorus-therian
keldeo-ordinary
keldeo-resolute
meloetta-aria
meloetta-pirouette
